<a href="https://colab.research.google.com/github/cwyalpha/Langchain-Chatchat-Colab/blob/main/Langchain_Chatchat_ChatGLM2_6b_int4_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
EMBEDDING_DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
# 首先确认输出是cuda，否则Runtime里类型要改成T4 GPU
print(EMBEDDING_DEVICE)

# 拉取仓库
!git clone https://github.com/THUDM/ChatGLM2-6B
%cd /content/ChatGLM2-6B
# int4版本需要安装其中的cpm_kernels
!pip install -r requirements.txt

# 基于Langchain-Chatchat 0.2.6测试通过
%cd /content/
!git clone https://github.com/chatchat-space/Langchain-Chatchat.git
%cd /content/Langchain-Chatchat/
!pip install -r requirements.txt
!pip install -r requirements_api.txt
!pip install -r requirements_webui.txt
!pip install --upgrade protobuf==3.19.6
!pip install gradio
!pip install accelerate
!pip install spacy
!pip install streamlit
!pip install pyngrok


# Download models
!git lfs install
!git clone https://huggingface.co/THUDM/chatglm2-6b-int4
!git clone https://huggingface.co/moka-ai/m3e-base

# 复制配置
%cd /content/Langchain-Chatchat/
# 复制配置文件
!python copy_config_example.py
# 修改model_config.py文件
!sed -i "s/THUDM\/chatglm2-6b-int4\"/\/content\/Langchain-Chatchat\/chatglm2-6b-int4\"/g" /content/Langchain-Chatchat/configs/model_config.py
!sed -i 's/\(LLM_MODEL\s*=\s*\)"chatglm2-6b"/\1"chatglm2-6b-int4"/' /content/Langchain-Chatchat/configs/model_config.py
!sed -i "s/moka-ai\/m3e-base/\/content\/Langchain-Chatchat\/m3e-base/g" /content/Langchain-Chatchat/configs/model_config.py
# 初始化知识库
!python init_database.py --recreate-vs
# 启动项目
!nohup python startup.py -a &

# 最后一步nohup运行之后，需要等待一段时间网页内才有内容
# 在/content/Langchain-Chatchat/nohup.out中看到0.0.0.0:8501后就可以正常访问
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
!npm install localtunnel
#!streamlit run app.py &>web_logs.txt &
!npx localtunnel --port 8501
# 在网页中输入IP后提交

cuda
Cloning into 'ChatGLM2-6B'...
remote: Enumerating objects: 280, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 280 (delta 143), reused 130 (delta 127), pack-reused 107
Receiving objects: 100% (280/280), 7.37 MiB | 22.45 MiB/s, done.
Resolving deltas: 100% (157/157), done.
/content/ChatGLM2-6B
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.1
    Uninstalling transformers-4.34.1:
      Successfully uninstalled transformers-4.34.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

  Using cached protobuf-4.24.4-cp37-abi3-manylinux2014_x86_64.whl (311 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.24.4 which is incompatible.
Updated git hooks.
Git LFS initialized.
fatal: destination path 'chatglm2-6b-int4' already exists and is not an empty directory.
fatal: destination path 'm3e-base' already exists and is not an empty directory.
/content/Langchain-Chatchat
/content/Langchain-Chatchat
database talbes reseted
recreating all vector stores
2023-10-30 03:01:33,297 - faiss_cache.py[line:75] - INFO: loading vector store in 'samples/vector_store' from disk.
{}
2023-10-30 03:01:33,382 - SentenceTra